# Dataset pre processing

This notbook focuses on the bncc dataset pre processing aiming the training of a classification model for "etapa do conhecimento" prediction

In [1]:
import html

import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn import (
    feature_extraction,
    linear_model,
    metrics,
    model_selection,
    preprocessing,
)

from src.data_preprocessing import cleaning

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Danilo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 1000)

## Selecting and filtering the features that will be used in the model

In [3]:
# Importing Data
df_bncc = pd.read_csv("C:/Users/Danilo/Desktop/Documentos Acadêmicos/TERA/TERAprojetos/Projeto final/BNCC/repo/bncc-classifier/data/raw/bncc_first_classifier.csv")

In [4]:
# Making a copy of the dataset and visualizing it
df_bncc_copy = df_bncc.copy()

df_bncc_copy.head()

,id,question,bulletType,name,slug,name.1,slug.1,name.2
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,2,História da Arte,historia-da-arte,Arte,arte,Fundamental II
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",2,Álgebra: Equações do 2º grau: Equação do 2º gr...,algebra-equacoes-do-2o-grau-equacao-do-2o-grau...,Matemática,matematica,Fundamental II
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,4,Outros,outros,Espanhol,espanhol,Fundamental II
3,106855,<p><div> A respeito dos serviços públicos e da...,2,Responsabilidade Civil do Estado,responsabilidade-civil-do-estado,Direito Administrativo,direito-administrativo,Concurso
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",2,Outros,outros,Química,quimica,Fundamental II


In [5]:
# Dropping unuseful columns
df_bncc_copy = df_bncc_copy[["id", "question", "name.2"]]

df_bncc_copy.head()

,id,question,name.2
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Fundamental II
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Fundamental II
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,Fundamental II
3,106855,<p><div> A respeito dos serviços públicos e da...,Concurso
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Fundamental II


In [6]:
# Renaming dataset columns
df_bncc_copy.columns = ["id", "questions", "target"]

df_bncc_copy.head(3)

,id,questions,target
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Fundamental II
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Fundamental II
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,Fundamental II


In [7]:
# Observing the classes of the target to drop, aiming to comprehend only the topics of bncc
df_bncc_copy["target"].value_counts().to_frame()

,target
Médio & Pré-Vestibular,35676
Fundamental II,33790
Fundamental I,18379
Concurso,9875
Militar,2200
OAB,80


In [8]:
df_bncc_copy.shape

(100000, 3)

In [9]:
df_bncc_copy_targets_bncc = df_bncc_copy[df_bncc_copy["target"].isin(["Médio & Pré-Vestibular", "Fundamental II", "Fundamental I"])]

In [10]:
# Observing if the filtering was correctly applied
df_bncc_copy_targets_bncc

,id,questions,target
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Fundamental II
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Fundamental II
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,Fundamental II
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Fundamental II
5,2016466,<p><strong>Anexe nesse espa&ccedil;o uma foto ...,Fundamental I
...,...,...,...
99994,2447693,<p>Leia a reportagem e responda a quest&atilde...,Fundamental I
99995,4330761,"<p>Para responder à questão, leia o trecho do ...",Médio & Pré-Vestibular
99996,3249373,"<h1 style=""list-style:none;margin:0px 0px 20px...",Fundamental II
99998,4150214,<p>Os machos e fêmeas de mosquitos do gênero <...,Fundamental II


In [11]:
# Observing if the filtering was correctly applied by visualizing the clases
df_bncc_copy_targets_bncc["target"].value_counts().to_frame()

,target
Médio & Pré-Vestibular,35676
Fundamental II,33790
Fundamental I,18379


In [12]:
# How many observations do we have now?
df_bncc_copy_targets_bncc.shape

(87845, 3)

In [13]:
# Encoding the target with labels for the classifier
# this procedure can be done with LabelEncoder from scikit-learn
# will give us more control over the pipeline

lb_enc = preprocessing.LabelEncoder()

# train on the column we want encode
lb_enc.fit(df_bncc_copy_targets_bncc["target"])

# transform the same column, but here we'll have this transformation for test and after for train
df_bncc_copy_targets_bncc["target_enc"] = lb_enc.transform(
    df_bncc_copy_targets_bncc["target"]
)
df_bncc_copy_targets_bncc

<ipython-input-13-e100d794caa5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc["target_enc"] = lb_enc.transform(


,id,questions,target,target_enc
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Fundamental II,1
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Fundamental II,1
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,Fundamental II,1
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Fundamental II,1
5,2016466,<p><strong>Anexe nesse espa&ccedil;o uma foto ...,Fundamental I,0
...,...,...,...,...
99994,2447693,<p>Leia a reportagem e responda a quest&atilde...,Fundamental I,0
99995,4330761,"<p>Para responder à questão, leia o trecho do ...",Médio & Pré-Vestibular,2
99996,3249373,"<h1 style=""list-style:none;margin:0px 0px 20px...",Fundamental II,1
99998,4150214,<p>Os machos e fêmeas de mosquitos do gênero <...,Fundamental II,1


In [14]:
# you can access how the labels was transformed by looking at the classes
lb_enc.classes_

array(['Fundamental I', 'Fundamental II', 'Médio & Pré-Vestibular'],
      dtype=object)

## Cleaning the dataset

In [15]:
# chaining all cleaning steps
df_bncc_copy_targets_bncc["questions_clean"] = (
    df_bncc_copy_targets_bncc["questions"]
    .astype(str)
    .apply(html.unescape)
    .apply(lambda x: cleaning.remove_html(x))
    .apply(lambda x: x.lower())
    .apply(lambda x: cleaning.remove_punctuation_2(x))
    .apply(cleaning.remove_italic_quotes)
    .apply(cleaning.remove_open_quotes)
    .apply(cleaning.remove_end_quotes)
    .apply(cleaning.remove_italic_dquotes)
    .apply(cleaning.remove_open_dquotes)
    .apply(cleaning.remove_quote)
    .apply(lambda x: cleaning.remove_pt_stopwords(x))
    .apply(lambda x: cleaning.remove_en_stopwords(x))
)

<ipython-input-15-8d5b5687da49>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bncc_copy_targets_bncc["questions_clean"] = (


In [16]:
df_bncc_copy_targets_bncc.head(10)

,id,questions,target,target_enc,questions_clean
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Fundamental II,1,base conhecimentos sobre compositor john cage ...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Fundamental II,1,achar número somado 2 igual inverso” equações ...
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,Fundamental II,1,2 complete espaços abaixo artigos determinados...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Fundamental II,1,chuva ácida fenômeno provocado poluição atmosf...
5,2016466,<p><strong>Anexe nesse espa&ccedil;o uma foto ...,Fundamental I,0,anexe nesse espaço foto ilustração
6,1744035,<p>Pagina nr. 54 do Livro did&aacute;tico.</p>,Fundamental II,1,pagina nr 54 livro didático
7,2525602,<p>O que &eacute; CHIAROSCURO?</p>,Médio & Pré-Vestibular,2,chiaroscuro
8,2610914,<p>Analise a igualdade a seguir</p><p><br></p>...,Fundamental II,1,analise igualdade seguiresta igualdade
10,2693547,<p> &Eacute; MUITO IMPORTANTE OBSERVAR TUDO CO...,Fundamental I,0,importante observar tudo muita atenção vezes o...
12,1961666,"<p style=""text-align:justify"">Jo&atilde;o prec...",Médio & Pré-Vestibular,2,joão precisa realizar movimentações financeira...


In [17]:
# class to remove frq and rare, we can choose how many rare or frq words to remove
remove_frq_rare = cleaning.RemoveFrqRare(df=df_bncc_copy_targets_bncc)
remove_frq_rare.calc_frq_words()
remove_frq_rare.calc_rare_words()
bncc_cleaned_df = remove_frq_rare.remove_frq_and_rare()

In [18]:
bncc_cleaned_df.head()

,id,questions,target,target_enc,questions_clean
0,3486670,<p>Com base em seus conhecimentos sobre o comp...,Fundamental II,1,base conhecimentos sobre compositor john cage ...
1,3343927,"<p>&ldquo;Achar um n&uacute;mero que, somado c...",Fundamental II,1,achar número somado 2 igual inverso” equações ...
2,1927600,<p>2. Complete os espaços abaixo com os artigo...,Fundamental II,1,2 complete espaços abaixo artigos determinados...
4,3691951,"<p style=""padding:0px;margin:10px 0px 0px;outl...",Fundamental II,1,chuva ácida fenômeno provocado poluição atmosf...
5,2016466,<p><strong>Anexe nesse espa&ccedil;o uma foto ...,Fundamental I,0,anexe nesse espaço foto ilustração


### Exporting the dataset

In [19]:
bncc_cleaned_df = bncc_cleaned_df[["id", "questions_clean", "target_enc"]]

In [20]:
# Exporting
bncc_cleaned_df.to_csv(
    "../../data/curated/df_bncc_model_two_curated_pedro_02032022.csv"
)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/curated/df_bncc_model_two_curated_pedro_02032022.csv'

## Modeling

In [ ]:
# removing registers with zero chars
bncc_cleaned_df["words_count"] = bncc_cleaned_df["questions_clean"].apply(len)
bncc_cleaned_filtered_df = bncc_cleaned_df[bncc_cleaned_df["words_count"] != 0]
bncc_cleaned_filtered_df

,id,questions_clean,target_enc,words_count
0,3486670,base conhecimentos sobre compositor john cage ...,1,346
1,3343927,achar número somado 2 igual inverso” equações ...,1,65
2,1927600,2 complete espaços abaixo artigos determinados...,1,359
4,3691951,chuva ácida fenômeno provocado poluição atmosf...,1,97
5,2016466,anexe nesse espaço foto ilustração,0,34
...,...,...,...,...
99994,2447693,leia reportagem responda questãosobre pergunta...,0,99
99995,4330761,responder questão leia trecho contoprefácio hi...,2,1458
99996,3249373,pais surgiu skate,1,17
99998,4150214,machos fêmeas mosquitos gênero aedes vivem loc...,1,415


Eu vi depois que essa poha do CountVectorizer n precisa tokenizar antes, pode jogar o texto mesmo q ele cria os tokens pro bag of words. Por isso q to dropando ae embaixo.

In [ ]:
# sklearn patterns names
X = bncc_cleaned_filtered_df["questions_clean"]
y = bncc_cleaned_filtered_df["target_enc"]
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, random_state=1
)

In [ ]:
# bow vect
vectBOW = feature_extraction.text.CountVectorizer()
X_train_trans = vectBOW.fit_transform(X_train)
X_test_trans = vectBOW.transform(X_test)
print(X_train_trans.shape, X_test_trans.shape)

(65094, 174497) (21698, 174497)


In [ ]:
logreg = linear_model.LogisticRegression(class_weight="balanced")
logreg.fit(X_train_trans, y_train)
y_pred_class = logreg.predict(X_test_trans)

C:\Users\Danilo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
print(metrics.classification_report(y_test, y_pred_class))

              precision    recall  f1-score   support

           0       0.65      0.78      0.71      4575
           1       0.65      0.65      0.65      8338
           2       0.77      0.70      0.74      8785

    accuracy                           0.70     21698
   macro avg       0.69      0.71      0.70     21698
weighted avg       0.70      0.70      0.70     21698



- Fundamental I - 0
- Fundamental II - 1
- Médio e Pré-Vestibular - 2